In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="50579563e8904db4a475c8c10654138c" 
secret = "b740dcff5ca9458289fa56a5f816870b"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
album_name = []
year = []

for i in range(0,2000, 50):
    album_results = sp.search(q='year:2020', type='albums', limit=50, offset=i)
    for i, t in enumerate(album_results['albums']['items']):
        artist_name.append(t['artists'][0]['name'])
        album_name.append(t['name'])
        year.append(t['release_date'])

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

HTTP Error for GET to https://api.spotify.com/v1/search returned 400 due to Bad search type field albums


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/search?q=year%3A2020&limit=50&offset=0&type=albums:
 Bad search type field albums

In [ ]:
import pandas as pd

df_albums = pd.DataFrame({'year':year,'artist_name':artist_name,'album_name':album_name})
df_albums.sort_values(by=['year'])

In [ ]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_albums.groupby(['artist_name','album_name'], as_index=True).size()
grouped[grouped > 1].count()

In [ ]:
df_albums.drop_duplicates(subset=['artist_name','album_name'], inplace=True)

In [ ]:
#se establece la conexión con google sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./Jupyter and Google Sheets-673752ccde13.json', scope)
gc = gspread.authorize(credentials)

In [ ]:
#aquí se específica qué google sheet vamos a tratar
spreadsheet_key = '1cMEmnIriNr-8a4bDykaZQuADyeNKYP72hGj93f-VMVw'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("spotify unified")
table = worksheet.get_all_values()

In [ ]:
from df2gspread import df2gspread as d2g
wks_name = '2020'
d2g.upload(df_albums, spreadsheet_key, wks_name, credentials=credentials, row_names=True)